In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [2]:
import kagglehub

# 1. Téléchargement et exploration du dataset
print("Téléchargement du dataset...")
path = kagglehub.dataset_download("kmader/malaria-bounding-boxes")
print(f"Dataset téléchargé dans: {path}")

c:\Users\rachid.mariane\OneDrive - Tersea\Bureau\YOLO\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Téléchargement du dataset...
Dataset téléchargé dans: C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1


In [3]:
# Définir les chemins des données
images_dir = os.path.join(path, "images")
annotations_file = os.path.join(path, "bounding_boxes.csv")

In [29]:
# Lire les annotations
#df_annotations = pd.read_csv(annotations_file)
#print(f"Nombre total d'annotations: {len(df_annotations)}")
#print(df_annotations.head())

#lire les annotations en format json
import json
import os

# Le chemin du fichier d'annotations
df_annotations = os.path.join(path, 'malaria', 'test.json')

# Ouvrir le fichier JSON et charger les annotations
with open(df_annotations, 'r') as f:
    annotations = json.load(f)

# Afficher un aperçu des données
print(f"Nombre total d'annotations: {len(annotations)}")
print("Aperçu des premières annotations:", annotations[:5])  # Affiche les 5 premières annotations

# 2. Vérifier le format du JSON et le transformer en DataFrame
if isinstance(annotations, dict) and "annotations" in annotations:
    df_annotations = pd.DataFrame(annotations["annotations"])
else:
    df_annotations = pd.DataFrame(annotations)
id_col = 'image_id'
if id_col not in df_annotations.columns:
    id_col = df_annotations.columns[0]  # Prend la première colonne comme fallback




Nombre total d'annotations: 120
Aperçu des premières annotations: [{'image': {'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2', 'pathname': '/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg', 'shape': {'r': 1383, 'c': 1944, 'channels': 3}}, 'objects': [{'bounding_box': {'minimum': {'r': 576, 'c': 1744}, 'maximum': {'r': 708, 'c': 1883}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 863, 'c': 1249}, 'maximum': {'r': 977, 'c': 1373}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 210, 'c': 1573}, 'maximum': {'r': 335, 'c': 1711}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 1017, 'c': 1124}, 'maximum': {'r': 1142, 'c': 1253}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 572, 'c': 1655}, 'maximum': {'r': 695, 'c': 1763}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 477, 'c': 1307}, 'maximum': {'r': 590, 'c': 1429}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 962,

In [30]:
# 2. Créer un Dataset personnalisé pour YOLO
class MalariaDataset(Dataset):
    def __init__(self, annotations_df, img_dir, transform=None):
        self.annotations = annotations_df
        self.img_dir = img_dir
        self.transform = transform
        self.image_ids = annotations_df['image_id'].unique()
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, f"{img_id}.png")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Récupérer toutes les annotations pour cette image
        img_annotations = self.annotations[self.annotations['image_id'] == img_id]
        
        # Dimensions de l'image
        height, width = image.shape[:2]
        
        # Préparer les boîtes englobantes au format YOLO [class_id, x_center, y_center, width, height]
        boxes = []
        for _, row in img_annotations.iterrows():
            # Convertir les coordonnées absolues en coordonnées relatives (format YOLO)
            x_min, y_min = row['x_min'], row['y_min']
            box_width = row['width']
            box_height = row['height']
            
            # Calculer le centre et normaliser
            x_center = (x_min + box_width / 2) / width
            y_center = (y_min + box_height / 2) / height
            # Normaliser la largeur et la hauteur
            norm_width = box_width / width
            norm_height = box_height / height
            
            # Ajouter la classe (0 = parasite de malaria)
            boxes.append([0, x_center, y_center, norm_width, norm_height])
        
        boxes = np.array(boxes)
        
        if self.transform:
            augmented = self.transform(image=image, bboxes=boxes)
            image = augmented['image']
            boxes = augmented['bboxes']
        
        return image, torch.tensor(boxes)


In [31]:
print("Colonnes disponibles :", df_annotations.columns.tolist())
print(df_annotations.head(3))


Colonnes disponibles : ['image', 'objects']
                                               image  \
0  {'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2...   
1  {'checksum': '2d85fc8ffdb875bb569878243437e4f5...   
2  {'checksum': 'f7c02b6247f3ccfcd4a22c1f06e0c72c...   

                                             objects  
0  [{'bounding_box': {'minimum': {'r': 576, 'c': ...  
1  [{'bounding_box': {'minimum': {'r': 1048, 'c':...  
2  [{'bounding_box': {'minimum': {'r': 979, 'c': ...  


In [32]:
print(df_annotations['image'].iloc[0])


{'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2', 'pathname': '/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg', 'shape': {'r': 1383, 'c': 1944, 'channels': 3}}


In [33]:
# Extraire le nom du fichier à partir de 'pathname' et créer une nouvelle colonne 'image_id'
df_annotations['image_id'] = df_annotations['image'].apply(lambda x: os.path.splitext(os.path.basename(x['pathname']))[0])



In [34]:
# Diviser les données en ensembles d'entraînement et de validation
train_ids, val_ids = train_test_split(df_annotations['image_id'].unique(), test_size=0.2, random_state=42)

train_df = df_annotations[df_annotations['image_id'].isin(train_ids)]
val_df = df_annotations[df_annotations['image_id'].isin(val_ids)]

print(f"Images d'entraînement: {len(train_ids)}")
print(f"Images de validation: {len(val_ids)}")


Images d'entraînement: 96
Images de validation: 24


In [35]:
# 4. Définir les transformations d'augmentation de données
try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    
    train_transform = A.Compose([
        A.Resize(416, 416),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    val_transform = A.Compose([
        A.Resize(416, 416),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    # Créer les datasets
    train_dataset = MalariaDataset(train_df, images_dir, transform=train_transform)
    val_dataset = MalariaDataset(val_df, images_dir, transform=val_transform)
    
    # Créer les dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=lambda x: x)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=lambda x: x)
    
except ImportError:
    print("Pour continuer, installez les bibliothèques requises:")
    print("pip install albumentations ultralytics")




In [36]:
# 5. Entraîner le modèle YOLO avec Ultralytics
def train_yolo():
    from ultralytics import YOLO
    
    # Créer un fichier de configuration YAML pour YOLOv8
    config_content = """
    # YOLOv8 Configuration
    path: {dataset_path}
    train: {train_path}
    val: {val_path}

    # Classes
    names:
      0: malaria_parasite
    """
    
    # Créer les répertoires nécessaires
    os.makedirs('yolo_data', exist_ok=True)
    os.makedirs('yolo_data/images/train', exist_ok=True)
    os.makedirs('yolo_data/images/val', exist_ok=True)
    os.makedirs('yolo_data/labels/train', exist_ok=True)
    os.makedirs('yolo_data/labels/val', exist_ok=True)
    
    # Préparer les données au format YOLO
    prepare_yolo_dataset(train_df, 'train')
    prepare_yolo_dataset(val_df, 'val')
    
    # Écrire le fichier de configuration
    with open('yolo_data/malaria.yaml', 'w') as f:
        f.write(config_content.format(
            dataset_path='yolo_data',
            train_path='images/train',
            val_path='images/val'
        ))
    
    # Charger le modèle YOLO préentraîné
    model = YOLO('yolov8n.pt')
    
    # Entraîner le modèle
    results = model.train(
        data='yolo_data/malaria.yaml',
        epochs=50,
        imgsz=416,
        batch=16,
        name='malaria_model'
    )
    
    return model, results

def prepare_yolo_dataset(annotations_df, subset):
    """
    Prépare les images et annotations au format YOLO
    """
    image_ids = annotations_df['image_id'].unique()
    
    for img_id in image_ids:
        # Copier l'image
        src_path = os.path.join(images_dir, f"{img_id}.png")
        dst_path = os.path.join(f'yolo_data/images/{subset}', f"{img_id}.png")
        
        if os.path.exists(src_path):
            # Lire l'image pour obtenir ses dimensions
            img = cv2.imread(src_path)
            height, width = img.shape[:2]
            
            # Copier l'image
            os.system(f"cp {src_path} {dst_path}")
            
            # Créer le fichier d'annotation YOLO
            img_annotations = annotations_df[annotations_df['image_id'] == img_id]
            
            with open(os.path.join(f'yolo_data/labels/{subset}', f"{img_id}.txt"), 'w') as f:
                for _, row in img_annotations.iterrows():
                    # Convertir au format YOLO: class_id x_center y_center width height
                    x_min, y_min = row['x_min'], row['y_min']
                    box_width, box_height = row['width'], row['height']
                    
                    x_center = (x_min + box_width / 2) / width
                    y_center = (y_min + box_height / 2) / height
                    norm_width = box_width / width
                    norm_height = box_height / height
                    
                    f.write(f"0 {x_center} {y_center} {norm_width} {norm_height}\n")


In [37]:
# 6. Visualiser les prédictions
def visualize_predictions(model, image_path):
    results = model.predict(image_path)
    
    # Afficher l'image avec les prédictions
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            conf = box.conf[0]
            cls = int(box.cls[0])
            
            plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                                             fill=False, edgecolor='red', linewidth=2))
            plt.text(x1, y1, f'Parasite: {conf:.2f}', color='white', 
                     backgroundcolor='red', fontsize=12)
    
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('prediction_example.png')
    plt.show()

In [44]:
import os

train_images_dir = 'yolo_data/images/train'
val_images_dir = 'yolo_data/images/val'

# Vérifier le contenu des répertoires d'images
print("Images d'entraînement:", os.listdir(train_images_dir))
print("Images de validation:", os.listdir(val_images_dir))


Images d'entraînement: []
Images de validation: []


In [51]:
path: r'C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria'  # Chemin vers le répertoire contenant les sous-dossiers 'images' et 'labels'
train: r'C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria\images'  # Répertoire des images d'entraînement
val: r'C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria\trainig.json'  # Répertoire des images de validation
nc: 1  # Nombre de classes
names: ['malaria']  # Liste des noms de classes


In [54]:
import os
from ultralytics import YOLO

# Définir les répertoires
images_dir = r'C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria\images'
data_yaml = r'C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria\trainig.json'  # Remplacez par le chemin réel de votre fichier YAML

# Fonction d'entraînement
def train_yolo():
    # Charger un modèle pré-entraîné
    model = YOLO('yolov8n.pt')  # Remplacez par le modèle de votre choix
    
    # Entraîner le modèle
    results = model.train(
        data=data_yaml,
        imgsz=640,  # Taille des images d'entrée
        epochs=10,  # Nombre d'époques
        batch=16,  # Taille du lot
        name='malaria_model'  # Nom du répertoire de résultats
    )
    return model, results

# Fonction de visualisation des prédictions
def visualize_predictions(model, img_path):
    results = model(img_path)
    results.show()  # Affiche les prédictions sur l'image

# Fonction principale
def main():
    print("Préparation des données...")
    
    try:
        print("Entraînement du modèle YOLO...")
        model, results = train_yolo()
        
        # Sélectionner une image pour visualiser les prédictions
        sample_img = os.path.join(images_dir, os.listdir(images_dir)[0])
        print(f"Visualisation des prédictions sur {sample_img}...")
        visualize_predictions(model, sample_img)
        
        print("Entraînement terminé! Le modèle est sauvegardé dans runs/detect/malaria_model")
        
    except ImportError:
        print("Pour entraîner le modèle YOLO, installez les bibliothèques requises : ultralytics")
        # Vous pouvez également ajouter une commande pour installer le package si nécessaire
        # os.system('pip install ultralytics')

if __name__ == "__main__":
    main()


Préparation des données...
Entraînement du modèle YOLO...
Ultralytics 8.3.104  Python-3.13.2 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria\trainig.json, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=malaria_model6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

RuntimeError: Dataset 'C://Users/rachid.mariane/.cache/kagglehub/datasets/kmader/malaria-bounding-boxes/versions/1/malaria/trainig.json' error  'C:\Users\rachid.mariane\.cache\kagglehub\datasets\kmader\malaria-bounding-boxes\versions\1\malaria\trainig.json' does not exist

NameError: name 'images' is not defined

In [ ]:
# 7. Fonction principale
def main():
    print("Préparation des données...")
    
    try:
        import ultralytics
        print("Entraînement du modèle YOLO...")
        model, results = train_yolo()
        
        # Sélectionner une image pour visualiser les prédictions
        sample_img = os.path.join(images_di, os.listdir(images_dir)[0])
        print(f"Visualisation des prédictions sur {sample_img}...")
        visualize_predictions(model, sample_img)
        
        print("Entraînement terminé! Le modèle est sauvegardé dans runs/detect/malaria_model")
        
    except ImportError:
        print("Pour entraîner le modèle YOLO, installez les bibliothèques requises:")
        

if __name__ == "__main__":
    main()